# 文本预处理 #
1. 将文本作为字符串加载到内存中。
1. 将字符串拆分为词元（如单词和字符）。
1. 建立一个词表，将拆分的词元映射到数字索引。
1. 将文本转换为数字索引序列，方便模型操作。

In [1]:
import collections
import re
from d2l import torch as d2l

In [2]:
#@save
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL+'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine(): #@save
    """将时间机器数据集加载到文本行的列表中"""
    with open(d2l.download('time_machine'),'r') as f:
        lines = f.readlines()
    # re.sub('[^A-Za-z]+',' ',line)：匹配非字母字符，替换为' '
    # strip()：去掉首尾空格
    # lower()：转换为小写
    return [re.sub('[^A-Za-z]+',' ',line).strip().lower() for line in lines]

lines = read_time_machine()
print(f'# 文本总数： {len(lines)}')
print(lines[0])
print(lines[10])

# 文本总数： 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


## 词元化 ##

In [3]:
def tokenize(lines, token='word'):
    """将文本行拆分为单词或字符标记"""
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('错误：未知词元类型：' + token)

tokens = tokenize(lines)
for i in range(10):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']


# 词表(vocabulary) #

In [4]:
class Vocab:  #@save
    """文本词表"""
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        # 按出现频率排序
        # 返回类似于Counter({'the': 2261, 'i': 1267, 'and': 1245, 'of': 1158, 'a': 816, 'to': 695, ...})
        counter = count_corpus(tokens)
        # 对词频的统计结果排序，输出排序好的键值对(字典的视图对象)
        # ".item()"：将字典转会为包含键值对的可迭代对象，每个对象是一个元组
        #   * .item()方法只适用于字典，类似还有.keys(), .values()
        # "key=lambda x: x[1]":按照每个元素的第二个元素（词频）排序
        # "reverse=True"：降序排序
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1],
                                   reverse=True)
        # 初始化idx_to_token列表（从索引到词元的映射列表）
        # 例：初始化为['<unk>', '<pad>', '<bos>', '<eos>']
        self.idx_to_token = ['<unk>'] + reserved_tokens
        # 初始化token_to_idx字典（从词元到索引的映射字典）
        # enumerate(self.idx_to_token)：生成带索引的迭代器，每次迭代返回(idx,token)元组
        # 例：初始化为{'<unk>'：0, '<pad>'：1, '<bos>'：2, '<eos>'：3}
        self.token_to_idx = {token: idx
                             for idx, token in enumerate(self.idx_to_token)}
        # 追加tokens中的词元到词表
        # 视图对象可以用for来遍历
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                # token追加到索引->词元的列表
                self.idx_to_token.append(token)
                # 'token':idx 键值对追加到映射字典
                self.token_to_idx[token] = len(self.idx_to_token) - 1
    
    def __len__(self):
        return len(self.idx_to_token)

    # 返回词元列表的索引列表
    # __getitem__是特殊方法，使得对象可以像列表一样通过索引运算符'[]'来访问元素
    def __getitem__(self, tokens):
        # 如果tokens不是list/tuple，即tokens是单个词元
        if not isinstance(tokens, (list, tuple)):
            # 在字典中查找tokens的索引，找不到则返回unk的索引
            # 这句使用了字典的get方法：dict.get(key,default)
            return self.token_to_idx.get(tokens, self.unk)
        # 递归地调用__getitem__函数，确保每个词元都找到索引
        return [self.__getitem__(token) for token in tokens]

    # 找到索引列表的词元列表
    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

    # "@property"装饰器将类中的函数变为属性，可以直接调用，不需要加括号
    @property
    def unk(self):  # 未知词元的索引为0
        return 0

    @property
    def token_freqs(self):
        return self._token_freqs

def count_corpus(tokens):  #@save
    """统计词元的频率"""
    # 这里的tokens是空列表或2D列表
    if len(tokens) == 0 or isinstance(tokens[0], list):
        # 将词元列表展平成一个列表
        # 这是一个嵌套列表推导式
        # 外层为：for line in tokens；内层为：for token in line
        # 等价于：
        #   flattened_tokens = []
        #   for line in tokens:
        #         for token in line:
        #             flattened_tokens.append(token)
        tokens = [token for line in tokens for token in line]
    # 返回的Counter对象是字典的子类，键为词元，值为词元的频率
    return collections.Counter(tokens)

In [5]:
# 创建Vocab对象
vocab = Vocab(tokens)
# 注意.item()返回的视图对象是一个可迭代对象，但不可以索引访问
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


In [6]:
for i in [0,10]:
    print('文本：', tokens[i])
    # 调用Vacob对象的__getitem__方法
    print("索引", vocab[tokens[i]])

文本： ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引 [1, 19, 50, 40, 2183, 2184, 400]
文本： ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
索引 [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


In [7]:
def load_corpus_time_machine(max_tokens=-1): #@save
    lines = read_time_machine()
    # 为了简化训练，用字符代替单词作为词元
    tokens = tokenize(lines,'char')
    vocab = Vocab(tokens)
    # 将所有文本行转化到一个大列表中（嵌套列表推导式）
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

# corpus是什么？是把文本的每个字母转化为索引后的索引列表
# vocab是什么？是一个Vocab对象，包含了字母的索引和字母的频率
corpus, vocab = load_corpus_time_machine()
corpus[:11], vocab['a','b','c','e']
len(corpus),len(vocab)

(170580, 28)